In [1]:
import sys
!{sys.executable} -m pip install strsimpy

In [9]:
import json
import strsimpy
import uuid
import datetime
from string import Template
from strsimpy import Jaccard
from tqdm import tqdm
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
def sparqlResultToDict(results):
    rows = []
    for result in results["results"]["bindings"]:
        row = {}
        for key in results["head"]["vars"]:
            if key in result:
                row[key] = result[key]["value"]
            else:
                row[key] = None
        rows.append(row)
    return rows

In [4]:
endpoint = "http://localhost:8887/blazegraph/sparql"    
sparql = SPARQLWrapper(endpoint, returnFormat=JSON)

In [5]:
labelsQuery = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX search: <https://platform.swissartresearch.net/search/>
SELECT ?subject ?label WHERE {
    ?subject a search:Object ;
        rdfs:label ?label
}
"""

In [6]:
# sparql.setQuery(labelsQuery)
# try:
#     ret = sparql.query().convert()
# except:
#     raise Exception("Could not execute query against endpoint", endpoint)
# labelsData = sparqlResultToDict(ret)

In [7]:
# with open('similarityResult.json', 'w') as fp:
#     json.dump(labelsData, fp, indent=4)

In [10]:
with open('similarityResult.json', 'r') as f:
    labelsData = json.load(f)

In [11]:
j = Jaccard(2)
cutoff = 0.5
maxSimilarCandidates = 100
for test in tqdm([d for d in labelsData if not 'candidates' in d]):
    test['candidates'] = []
    testLabel = test['label']
    for candidate in labelsData:
        if candidate != test:
            similarity = j.similarity(testLabel, candidate['label'])
            if similarity > cutoff:
                test['candidates'].append({
                    'subject': candidate['subject'],
                    'label': candidate['label'],
                    'similarity': similarity
                })
        if len(test['candidates']) > maxSimilarCandidates:
            continue
    with open('similarityResult.json', 'w') as fp:
        json.dump(labelsData, fp, indent=4)

0it [00:00, ?it/s]


In [12]:
# import json
# with open('similarityResult.json', 'w') as fp:
#     json.dump(labelsData, fp, indent=4)